# Cloudinary Upload Smoke Test

Prereqs in `.env`:
- `CLOUDINARY_CLOUD_NAME`
- `CLOUDINARY_API_KEY`
- `CLOUDINARY_API_SECRET` (for signed uploads) **or** `CLOUDINARY_UPLOAD_PRESET` (unsigned preset)
- Optional: `CLOUDINARY_PUBLIC_ID_PREFIX` (default: plots/)

Run the cell to generate a simple plot in the Daytona sandbox and upload it directly to Cloudinary. The tool output should include the public URL.

In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# Load .env manually to ensure creds are present in the notebook kernel
env_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".env"))
if os.path.exists(env_path):
    with open(env_path) as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#") or "=" not in line:
                continue
            k, v = line.split("=", 1)
            os.environ.setdefault(k, v)

from tools.code_execution import execute_python_code

code = r'''
import numpy as np
import matplotlib.pyplot as plt
x = np.linspace(0, 2*np.pi, 200)
plt.figure(figsize=(8,4))
plt.plot(x, np.sin(x), label="sin(x)")
plt.plot(x, np.cos(x), label="cos(x)")
plt.legend()
plt.title("Cloudinary Upload Smoke Test")
plt.tight_layout()
plt.savefig('/home/daytona/outputs/cloudinary_smoke.png', dpi=120, bbox_inches='tight')
plt.close()
'''

result = execute_python_code(code)
print(result)
# Extract URL if present and test retrieval
url = None
for line in str(result).splitlines():
    if "http" in line and "cloudinary" in line:
        url = line.split()[-1]
        print("\nFetched URL to verify:", url)
        break

if url:
    import requests
    resp = requests.get(url)
    print("GET status:", resp.status_code, "content-type:", resp.headers.get("Content-Type"))
else:
    print("No URL found in output; check warnings above.")

Generated files: cloudinary_smoke.png

Plot URLs: https://res.cloudinary.com/dsiwt2ptv/image/upload/v1766512383/plots/ced77f2798dd47bcb45910be4ebc4719_cloudinary_smoke.png

Fetched URL to verify: https://res.cloudinary.com/dsiwt2ptv/image/upload/v1766512383/plots/ced77f2798dd47bcb45910be4ebc4719_cloudinary_smoke.png
GET status: 200 content-type: image/png
